## Basic Assignment

This is applying the most basic scenario.

## Setup

We have to get the data into the proper format which will determine the results.

In [102]:
import openpyxl
import pandas as pd
import numpy as np

from pathlib import Path

In [78]:
filename = './bank_schedule.xlsx'
filepath = Path(filename)

file = pd.read_excel(filepath, None)
sheets = list(file.keys())
sheets

['Heatmap', 'Resource', 'Activity']

In [79]:
dfHeatmap = pd.read_excel(
    open(filepath, 'rb'), 
    sheet_name=sheets[0]
    )
print(f'shape: {dfHeatmap.shape}')
print(f'fields: {dfHeatmap.columns}')

shape: (10, 2)
fields: Index(['Bank', 'Priority'], dtype='object')


In [80]:
dfResource = pd.read_excel(
    open(filepath, 'rb'), 
    sheet_name=sheets[1]
    )
print(f'shape: {dfResource.shape}')
print(f'fields: {dfResource.columns}')

shape: (15, 3)
fields: Index(['Resource', 'Experience', 'Weeks Not Available'], dtype='object')


In [81]:
dfActivity = pd.read_excel(
    open(filepath, 'rb'), 
    sheet_name=sheets[2]
    )
print(f'shape: {dfActivity.shape}')
print(f'fields: {dfActivity.columns}')

shape: (10, 3)
fields: Index(['Activity', 'Bank', 'Weeks Needed'], dtype='object')


## Prepare Data

* separate activitiy schedule into time spans
* for each time span
  - remove people not available for time span
  - determine cost for each person-job: experience
  - ...

In [82]:
dfActivity['Weeks Needed'][0]

'13, 17, 18'

In [83]:
dfActivity['WeeksNeeded'] = dfActivity['Weeks Needed'].apply(lambda x: [int(item) for item in x.split(',')])

In [84]:
dfActivity[['Weeks Needed', 'WeeksNeeded']].head(3)

,Weeks Needed,WeeksNeeded
0,"13, 17, 18","[13, 17, 18]"
1,"06, 14, 15, 16, 17","[6, 14, 15, 16, 17]"
2,"19, 20, 21","[19, 20, 21]"


In [85]:
df = dfActivity.merge(dfHeatmap, on='Bank', how='outer')

In [94]:
df.head(3)

,Activity,Bank,Weeks Needed,WeeksNeeded,Priority
0,Activity-1,Bank-1,"13, 17, 18","[13, 17, 18]",1
1,Activity-10,Bank-10,"14, 15, 16","[14, 15, 16]",10
2,Activity-2,Bank-2,"06, 14, 15, 16, 17","[6, 14, 15, 16, 17]",2


In [86]:
sWeeks = set()
for x in df['WeeksNeeded'].to_list():
    for item in x:
        #new_item = '0'+str(item) if len(str(item)) < 2 else str(item)
        sWeeks.add(item)

lWeeks = []
for item in sWeeks:
    new_item = '0'+str(item) if len(str(item)) < 2 else str(item)
    lWeeks.append(new_item)


#review
weeks = list(sWeeks)
print(f'weeks: {weeks}')
print(f'length: {len(weeks)}')

weeks = list(lWeeks)
print(f'weeks: {weeks}')
print(f'length: {len(weeks)}')

weeks: [6, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
length: 14
weeks: ['06', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25']
length: 14


## Preprocess

* remove people not available for time span
* determine cost for each person-job: experience


We have the following variables:

* experience is a linear measure
* 

We want a cost function that meets the following criteria:

* hightest-priority (1) - highest-experience (1) = no cost (0)
* hightest-priority (10) - lowest-experience (1) = cost (10)
* lowest-priority (10) - highest-experience (1) = cost (10)
* lowest-priority (10) - lowest-experience (10) = no cost (0)

This equates to the absolute value of the difference between the two

In [93]:
dfResource.head(3)

,Resource,Experience,Weeks Not Available
0,Austin Powers,1,"06, 09"
1,Dr. Evil,2,"23, 33"
2,Basil Exposition,3,"19,"


In [92]:
#using int week: df[pd.Series(weeks[0]).isin(df['WeeksNeeded'])]

#using str week
for week in weeks:
    tmp_job = df[df['Weeks Needed'].str.contains(week)]
    tmp_people = dfResource[~dfResource['Weeks Not Available'].str.contains(week)]
    print(f'week {week} has {tmp_people.shape[0]} people to assign to {tmp_job.shape[0]} jobs')

week 06 has 14 people to assign to 1 jobs
week 13 has 15 people to assign to 1 jobs
week 14 has 13 people to assign to 2 jobs
week 15 has 13 people to assign to 3 jobs
week 16 has 15 people to assign to 4 jobs
week 17 has 14 people to assign to 4 jobs
week 18 has 14 people to assign to 4 jobs
week 19 has 11 people to assign to 3 jobs
week 20 has 15 people to assign to 3 jobs
week 21 has 15 people to assign to 2 jobs
week 22 has 15 people to assign to 1 jobs
week 23 has 13 people to assign to 2 jobs
week 24 has 15 people to assign to 2 jobs
week 25 has 15 people to assign to 2 jobs


In [59]:
df['Weeks Needed'].iloc[2]

'6, 14, 15, 16, 17'

In [ ]:
#TODO: get person name, assignment for each

In [141]:
tmp_job

,Activity,Bank,Weeks Needed,WeeksNeeded,Priority
1,Activity-10,Bank-10,"14, 15, 16","[14, 15, 16]",10
2,Activity-2,Bank-2,"06, 14, 15, 16, 17","[6, 14, 15, 16, 17]",2
8,Activity-8,Bank-8,"15, 16, 17, 18, 19","[15, 16, 17, 18, 19]",8
9,Activity-9,Bank-9,"16, 17, 18","[16, 17, 18]",9


In [142]:
tmp_people

,Resource,Experience,Weeks Not Available
0,Austin Powers,1,"06, 09"
1,Dr. Evil,2,"23, 33"
2,Basil Exposition,3,"19,"
3,Mrs. Kensington,4,14
4,Number Two,5,"08, 15"
5,Scott Evil,6,"17, 19"
6,Frau Farbissina,7,"08, 09"
7,Patty O'Brien,8,"15,"
8,Cmdr. Gilmour,9,18
9,Mustafa,10,19


In [98]:
week = '16'
tmp_job = df[df['Weeks Needed'].str.contains(week)]
tmp_people = dfResource[~dfResource['Weeks Not Available'].str.contains(week)]

task = []
for jobPrior in tmp_job['Priority'].to_list():
    worker = []
    for personExp in tmp_people['Experience'].to_list():
        diff = abs( jobPrior - personExp )
        worker.append(diff)
    task.append(worker)


In [108]:
costs = np.array(task).transpose()
costs

array([[ 9,  1,  7,  8],
       [ 8,  0,  6,  7],
       [ 7,  1,  5,  6],
       [ 6,  2,  4,  5],
       [ 5,  3,  3,  4],
       [ 4,  4,  2,  3],
       [ 3,  5,  1,  2],
       [ 2,  6,  0,  1],
       [ 1,  7,  1,  0],
       [ 0,  8,  2,  1],
       [ 1,  9,  3,  2],
       [ 2, 10,  4,  3],
       [ 3, 11,  5,  4],
       [ 4, 12,  6,  5],
       [ 5, 13,  7,  6]])

In [109]:
num_workers = len(costs)
num_tasks = len(costs[0])

print(f'Number of workers: {num_workers}')
print(f'Number of tasks: {num_tasks}')

Number of workers: 15
Number of tasks: 4


## Apply Optimization

In [110]:
from ortools.linear_solver import pywraplp

In [113]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver("SCIP")

if not solver:
    print('Error')

In [114]:
# x[i, j] is an array of 0-1 variables, which will be 1
# if worker i is assigned to task j.
x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.IntVar(0, 1, "")

In [115]:
# Each worker is assigned to at most 1 task.
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) <= 1)

# Each task is assigned to exactly one worker.
for j in range(num_tasks):
    solver.Add(solver.Sum([x[i, j] for i in range(num_workers)]) == 1)

In [138]:
# Create the objective function
objective_terms = []
for i in range(num_workers):
    for j in range(num_tasks):
        diff = costs[i][j] - x[i, j]
        result = diff if diff >= 0 else -1 * diff
        objective_terms.append( diff )
solver.Minimize(solver.Sum(objective_terms))

In [139]:
# Invoke solver
print(f"Solving with {solver.SolverVersion()}")
status = solver.Solve()

Solving with SCIP 8.0.4 [LP solver: Glop 9.8]


In [146]:
# Print solution
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f"Total cost = {solver.Objective().Value()}\n")
    for i in range(num_workers):
        for j in range(num_tasks):
            # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
            if x[i, j].solution_value() > 0.5:
                print(f"Worker {i+1} ({tmp_people['Resource'].to_list()[i]}) assigned to task {j+1} ({tmp_job['Activity'].to_list()[j]}):" + f" Cost: {costs[i][j]}")
else:
    print("No solution found.")

Total cost = 261.0

Worker 2 (Dr. Evil) assigned to task 2 (Activity-2): Cost: 0
Worker 8 (Patty O'Brien) assigned to task 3 (Activity-8): Cost: 0
Worker 9 (Cmdr. Gilmour) assigned to task 4 (Activity-9): Cost: 0
Worker 10 (Mustafa) assigned to task 1 (Activity-10): Cost: 0
